In [1]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 42.6 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pylibcudf-cu12=

In [ ]:
import pandas as pd
import pandas as pd
import numpy as np
import collections
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, DefaultDataCollator
import evaluate
import os
import shutil
from IPython.display import FileLink


2025-12-12 17:22:53.764458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765560173.974255      20 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765560174.038437      20 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

In [3]:
from transformers import EarlyStoppingCallback

In [ ]:
MODEL_CHECKPOINT = "google-bert/bert-base-multilingual-cased"

model = AutoModelForQuestionAnswering.from_pretrained(MODEL_CHECKPOINT)

tokenizer = AutoTokenizer.from_pretrained(MODEL_CHECKPOINT)
max_length = 512    
doc_stride = 128    

In [ ]:
# Tìm vị trí ký tự bắt đầu
def find_start_index(row):
    return row['context'].find(row['answer'])

def prepare_train_features(examples):
    examples["question"] = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=max_length,
        stride=doc_stride,
        truncation="only_second",
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )
    sample_mapping = inputs.pop("overflow_to_sample_mapping")
    offset_mapping = inputs.pop("offset_mapping")
    inputs["start_positions"] = []
    inputs["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        sample_index = sample_mapping[i]
        answers = examples["answer"][sample_index]
        start_char = examples["answer_start"][sample_index]
        end_char = start_char + len(answers)
        sequence_ids = inputs.sequence_ids(i)

        # Tìm ngữ cảnh (context)
        idx = 0
        while sequence_ids[idx] != 1: idx += 1
        context_start = idx
        while sequence_ids[idx] == 1: idx += 1
        context_end = idx - 1

        if offsets[context_start][0] > start_char or offsets[context_end][1] < end_char:
            inputs["start_positions"].append(0)
            inputs["end_positions"].append(0)
        else:
            idx = context_start
            while idx <= context_end and offsets[idx][0] <= start_char: idx += 1
            inputs["start_positions"].append(idx - 1)
            idx = context_end
            while idx >= context_start and offsets[idx][1] >= end_char: idx -= 1
            inputs["end_positions"].append(idx + 1)
    return inputs

def prepare_validation_features(examples):
    examples["question"] = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        max_length=max_length,
        stride=doc_stride,
        truncation="only_second",
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    sample_mapping = inputs.pop("overflow_to_sample_mapping")
    
    inputs["example_id"] = []

    for i in range(len(inputs["input_ids"])):
        sample_index = sample_mapping[i]
        inputs["example_id"].append(examples["id"][sample_index])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)
        ]

    return inputs

In [ ]:
df = pd.read_csv("/kaggle/input/data.csv")
df['id'] = df.index.astype(str)
df.head()


,Unnamed: 0,context,question,answer,id
0,0,"Tôi xin hỏi, một xã giai đoạn 2016-2020 là xã ...",Người dân tộc thiểu số xã nông thôn mới có đượ...,Bộ Y tế trả lời vấn đề này như sau:Ngày 19/10/...,0
1,1,Tôi đóng BHXH từ tháng 6/2011 đến tháng 12/201...,Có được truy lĩnh chế độ thai sản?,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...,1
2,2,"Tôi xin hỏi, trên giấy ra viện có ghi chú ngườ...",Khi nào cần xin thêm Giấy chứng nhận nghỉ việc...,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...,2
3,3,"Tôi năm nay 53 tuổi, đóng BHXH tự nguyện được ...",Đóng BHXH tự nguyện bao lâu thì được lương hưu?,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...,3
4,4,Tôi đóng BHXH từ tháng 1/2020 đến tháng 12/202...,Đóng BHXH bao lâu trước khi sinh thì được hưởn...,Bảo hiểm xã hội Việt Nam trả lời vấn đề này nh...,4


In [ ]:
df['answer_start'] = df.apply(find_start_index, axis=1)
raw_datasets = Dataset.from_pandas(df)

raw_datasets = raw_datasets.train_test_split(test_size=0.2)

In [7]:
raw_datasets["train"].column_names

['Unnamed: 0', 'context', 'question', 'answer', 'id', 'answer_start']

In [9]:
train_dataset = raw_datasets["train"].map(
    prepare_train_features,
    batched=True,
    remove_columns=raw_datasets["train"].column_names
)

validation_dataset = raw_datasets["test"].map(
    prepare_train_features,
    batched=True,
    remove_columns=raw_datasets["test"].column_names
)

Map:   0%|          | 0/25860 [00:00<?, ? examples/s]

Map:   0%|          | 0/6465 [00:00<?, ? examples/s]

In [ ]:
args = TrainingArguments(
    output_dir="bert-legal-qa",
    
    eval_strategy="epoch",  
    save_strategy="epoch",  
    
    logging_strategy="steps", 
    logging_steps=50,       
    
    load_best_model_at_end=True,       
    metric_for_best_model="eval_loss", 
    greater_is_better=False,          
    save_total_limit=1,                
    
    learning_rate=3e-5,
    num_train_epochs=20,    
    weight_decay=0.01,
    per_device_train_batch_size=8, 
    per_device_eval_batch_size=8,
    report_to="none",      
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset, 
    processing_class=tokenizer,
    data_collator=DefaultDataCollator(),
    

    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

print("--- BẮT ĐẦU TRAIN (EVAL THEO EPOCH) ---")
trainer.train()

--- BẮT ĐẦU TRAIN (EVAL THEO EPOCH) ---


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.106000,0.092797
2,0.064200,0.097458
3,0.048400,0.072623
4,0.055600,0.081258
5,0.045600,0.079025
6,0.024400,0.097305


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

TrainOutput(global_step=13590, training_loss=0.07637731325459708, metrics={'train_runtime': 15802.1281, 'train_samples_per_second': 45.863, 'train_steps_per_second': 2.867, 'total_flos': 5.681166344305459e+16, 'train_loss': 0.07637731325459708, 'epoch': 6.0})

In [ ]:
checkpoint_path = "/kaggle/working/bert-legal-qa"
output_zip_name = "legal_qa_model_lite.zip"

files_to_delete = [
    "optimizer.pt"
]

print(f"Đang dọn dẹp thư mục: {checkpoint_path}...")
for filename in files_to_delete:
    file_path = os.path.join(checkpoint_path, filename)
    if os.path.exists(file_path):
        os.remove(file_path)
        print(f"Đã xóa: {filename}")
    else:
        print(f"Không tìm thấy (hoặc đã xóa): {filename}")

print(f"\nĐang nén thư mục thành {output_zip_name}...")
shutil.make_archive("legal_qa_model_lite", 'zip', checkpoint_path)

Đang dọn dẹp thư mục: /kaggle/working/bert-legal-qa...
Không tìm thấy (hoặc đã xóa): optimizer.pt

Đang nén thư mục thành legal_qa_model_lite.zip...


'/kaggle/working/legal_qa_model_lite.zip'